 # This script reads in an AVIRIS netCDF from BioScape and selects specific wavelengths and saves em as a geotiff for easy arcgis access

#### cami pawlak 7/14/2025

#### First import packages

In [35]:
import netCDF4 as nc
import xarray as xr
import rioxarray
from affine import Affine
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from rasterio.enums import Resampling
import numpy as np
from pathlib import Path

#### Read in the dataset and just see some of the info about it

In [37]:
### Set fn once here:
#fn = "ang20231029t114806_014_L2A_OE_0b4f48b4_RFL_ORT.nc"
fn = "ang20231109t074932_002_L2A_OE_0b4f48b4_RFL_ORT.nc"
#fn = "ang20231126t104855_017_L2A_OE_0b4f48b4_RFL_ORT.nc"
ds = nc.Dataset(fn)

In [38]:
print(ds)

<class 'netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2024-11-27T01:35:58Z
    summary: The Airborne Visible / Infrared Imaging Spectrometer - Next Generation (AVIRIS-NG) is part of NASA’s Airborne Science Program (ASP) and the Jet Propulsion Laboratory’s (JPL) Earth Science Airborne Program. AVIRIS-NG is the successor to AVIRIS-Classic and provides high signal-to-noise ratio imaging spectroscopy measurements in 425 contiguous spectral channels with wavelengths in the solar reflected spectral range (380-2510 nm).
    keywords: Imaging Spectroscopy, AVIRIS, AVIRIS-NG
    sensor: Airborne Visible / Infrared Imaging Spectrometer Next Generation
    instrument: AVIRIS-NG
    platform: Gulfstream III
    institution: NASA Jet Propulsion Laboratory/California Institute of Technology
    keywords_vocabulary: NASA Global Change Master Directory (GCMD) Science Keywords
    creator_name: Jet Propulsion Laboratory/California Institu

#### The code below is all the necessary steps- change fn to your file of interest

In [ ]:
# below this works.

In [61]:
import xarray as xr
import rioxarray
import numpy as np
import os

# --- Configuration ---
netcdf_file_path = 'ang20231126t104855_017_L2A_OE_0b4f48b4_RFL_ORT.nc'

# Dynamically generate the output filename
base_filename = os.path.splitext(os.path.basename(netcdf_file_path))[0]
output_stacked_geotiff_name = f"{base_filename}_stacked.tif"

output_geotiff_dir = 'output_geotiffs'

target_wvls = np.array([
    467.35565, 472.35565, 477.36566,
    537.47565, 542.48566, 547.48566,
    637.64560, 642.66564, 647.67565,
    697.74567, 702.75570, 717.78564,
    827.97565, 832.98566, 837.99567
])

os.makedirs(output_geotiff_dir, exist_ok=True)

try:
    # Open the root dataset FIRST to get the correct easting/northing coordinates
    ds_root = xr.open_dataset(netcdf_file_path)

    # Open the 'reflectance' group dataset
    ds_reflectance_group = xr.open_dataset(netcdf_file_path, group='reflectance')
    reflectance_data = ds_reflectance_group['reflectance']

    print(f"\nDimensions of reflectance data initially: {reflectance_data.dims}")
    print(f"Coordinates of reflectance data initially: {reflectance_data.coords}")

    # --- Assign real-world easting/northing coordinates from ds_root ---
    if 'easting' in ds_root.coords and 'northing' in ds_root.coords:
        reflectance_data = reflectance_data.assign_coords(
            easting=ds_root.coords['easting'],
            northing=ds_root.coords['northing']
        )
        print("\nSuccessfully assigned real-world easting/northing coordinates from root dataset.")
        print(f"First 'easting' after assignment: {reflectance_data['easting'].values[0]}")
        print(f"Last 'easting' after assignment: {reflectance_data['easting'].values[-1]}")
        print(f"First 'northing' after assignment: {reflectance_data['northing'].values[0]}")
        print(f"Last 'northing' after assignment: {reflectance_data['northing'].values[-1]}")
    else:
        print("\nERROR: Real-world 'easting' or 'northing' coordinate variables not found in root dataset. Cannot proceed with georeferencing.")
        raise ValueError("Spatial coordinates not found as expected in root dataset.")

    # --- Handle Dimension Renaming for Robustness ---
    reflectance_data = reflectance_data.rename({'easting': 'x', 'northing': 'y'})
    print(f"Dimensions after renaming: {reflectance_data.dims}")


    # --- Check and Apply Y-axis (northing) order (if needed) ---
    first_y = reflectance_data['y'].values[0]
    last_y = reflectance_data['y'].values[-1]
    print(f"\nFirst 'y' (northing) value: {first_y}")
    print(f"Last 'y' (northing) value: {last_y}")

    if first_y < last_y: # If first is LESS than last, it's increasing, so flip
        print("Warning: 'y' (northing) coordinates are increasing. Flipping the 'y' dimension.")
        reflectance_data = reflectance_data.reindex(y=list(reversed(reflectance_data.y)))
        print(f"First 'y' (northing) value after flip: {reflectance_data['y'].values[0]}")
        print(f"Last 'y' (northing) value after flip: {reflectance_data['y'].values[-1]}")
    else:
        print("'y' (northing) coordinates are already decreasing or stable. No flip needed for Y axis.")


    # Now set spatial dimensions using 'x' and 'y'.
    reflectance_data = reflectance_data.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)


    # --- CRS Handling ---
    actual_crs_wkt = None
    if 'transverse_mercator' in ds_root.variables:
        tm_var = ds_root.variables['transverse_mercator']
        if 'crs_wkt' in tm_var.attrs:
            actual_crs_wkt = tm_var.attrs['crs_wkt']
        elif 'spatial_ref' in tm_var.attrs:
            actual_crs_wkt = tm_var.attrs['spatial_ref']
    
    if actual_crs_wkt:
        reflectance_data.rio.write_crs(actual_crs_wkt, inplace=True)
        print(f"\nCRS applied from WKT: {reflectance_data.rio.crs}")
    else:
        print("\nCRITICAL: No CRS WKT could be automatically determined. Using hardcoded EPSG:32734.")
        reflectance_data.rio.write_crs("EPSG:32734", inplace=True) # Fallback to known CRS
        print("Manually assigned CRS as: EPSG:32734")


    # --- Verify Geotransform (for debugging) ---
    calculated_transform = reflectance_data.rio.transform()
    if calculated_transform:
        print(f"\nCalculated Geotransform: {calculated_transform}")
        if calculated_transform.f > 0:
            print("WARNING: Y pixel size in geotransform is positive. This is unexpected for GeoTIFF top-left origin with decreasing Y coordinates.")
    else:
        print("\nWarning: No geotransform could be calculated by rioxarray despite having spatial coordinates.")


    # --- Determine the nodata value from the original data ---
    output_nodata_value = None
    if '_FillValue' in reflectance_data.encoding:
        output_nodata_value = reflectance_data.encoding['_FillValue']
        print(f"Determined nodata value from encoding: {output_nodata_value}")
    elif '_FillValue' in reflectance_data.attrs:
        output_nodata_value = reflectance_data.attrs['_FillValue']
        print(f"Determined nodata value from attributes: {output_nodata_value}")
    else:
        output_nodata_value = np.nan
        print("No explicit _FillValue found in original data encoding or attributes. Will use NaN for nodata.")


    # --- Select all target bands in one go ---
    stacked_bands = reflectance_data.sel(wavelength=target_wvls, method='nearest').copy()

    # --- Clean _FillValue from the stacked_bands DataArray ---
    if '_FillValue' in stacked_bands.encoding:
        del stacked_bands.encoding['_FillValue']
        print(f"Cleaned _FillValue from stacked_bands.encoding.")
    if '_Netcdf4FillValue' in stacked_bands.encoding:
        del stacked_bands.encoding['_Netcdf4FillValue']
        print(f"Cleaned _Netcdf4FillValue from stacked_bands.encoding.")
    if '_FillValue' in stacked_bands.attrs:
        del stacked_bands.attrs['_FillValue']
        print(f"Cleaned _FillValue from stacked_bands.attrs.")

    # --- Set the nodata value for the multi-band output ---
    stacked_bands.rio.write_nodata(output_nodata_value, inplace=True)
    print(f"Set nodata for stacked GeoTIFF to: {output_nodata_value}")

    # --- Custom Band Naming: Modify the 'wavelength' coordinate values ---
    # Create a list of string names like 'wv_467' based on the actual wavelengths
    actual_wvls = stacked_bands.wavelength.values
    band_string_names = [f"wv_{int(np.round(wvl))}" for wvl in actual_wvls]
    
    # Assign these string names back to the 'wavelength' coordinate
    # This will make the coordinate values be strings, not numbers.
    stacked_bands['wavelength'] = band_string_names
    
    # Now, rename the 'wavelength' dimension to 'band'.
    # Since the 'wavelength' coordinate now holds strings, the 'band' coordinate
    # will also hold these strings, which rioxarray uses for band names.
    stacked_bands = stacked_bands.rename({"wavelength": "band"})
    
    print(f"Dimensions after renaming wavelength to band: {stacked_bands.dims}")
    print(f"Band coordinate values (names): {stacked_bands.band.values}")

    # --- Export the stacked GeoTIFF ---
    output_filepath = os.path.join(output_geotiff_dir, output_stacked_geotiff_name)
    
    stacked_bands.rio.to_raster(
        output_filepath,
        driver="GTiff",
        compress="LZW",
        tiled=True,
        blockxsize=256,
        blockysize=256,
        # nodata is already set via .rio.write_nodata() on the DataArray
    )

    print(f"\nSuccessfully stacked and saved {len(target_wvls)} bands to: {output_filepath}")
    print(f"Output GeoTIFF name: {output_stacked_geotiff_name}")

    ds_root.close()
    ds_reflectance_group.close()

except FileNotFoundError:
    print(f"Error: NetCDF file not found at '{netcdf_file_path}'. Please check the path.")
except KeyError as e:
    print(f"Error accessing data: {e}. Make sure 'reflectance' group and variable exist as expected.")
    print("If you're still seeing issues, you might need to manually inspect the NetCDF structure with `ncdump -h your_file.nc` to confirm variable/group names.")
except ValueError as e:
    print(f"A configuration error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Dimensions of reflectance data initially: ('wavelength', 'northing', 'easting')
Coordinates of reflectance data initially: Coordinates:
  * wavelength  (wavelength) float32 2kB 377.2 382.2 ... 2.496e+03 2.501e+03

Successfully assigned real-world easting/northing coordinates from root dataset.
First 'easting' after assignment: 812323.871455526
Last 'easting' after assignment: 813857.171455526
First 'northing' after assignment: 6201548.30075942
Last 'northing' after assignment: 6197100.400759419
Dimensions after renaming: ('wavelength', 'y', 'x')

First 'y' (northing) value: 6201548.30075942
Last 'y' (northing) value: 6197100.400759419
'y' (northing) coordinates are already decreasing or stable. No flip needed for Y axis.

CRS applied from WKT: EPSG:32733

Calculated Geotransform: | 1.90, 0.00, 812322.92|
| 0.00,-1.90, 6201549.25|
| 0.00, 0.00, 1.00|
Determined nodata value from encoding: -9999.0
Cleaned _FillValue from stacked_bands.encoding.
Set nodata for stacked GeoTIFF to: -9999.0